1. Wprowadzenie

    Celem projektu jest przewidzenie bazowej ceny oferty noclegowej (base_price) na podstawie statycznych cech obiektu oraz informacji historycznych o jakości i popularności oferty.Bazowa cena rozumiana jest jako typowa cena za noc, niezależna od konkretnej daty 

    W projekcie przygotowano dwa modele regresyjne do porównania:
        Model liniowy: Ridge Regression
        Model nieliniowy: CatBoost Regressor

    Takie podejście pozwala porównać:
        model prosty, interpretowalny (baseline),
        model nieliniowy, zdolny uchwycić złożone relacje między cechami.

2. Wykorzystane cechy
Do predykcji wykorzystano następujące grupy cech:
    1. Cechy obiektu : room_type, property_type, accommodates, bedrooms, beds, bathrooms, amenities_count, minimum_nights, maximum_nights
    2. Lokalizacja : city, neighbourhood_cleansed, latitude, longitude
    3. Popularność i aktywność : number_of_reviews, reviews_per_month, review_count, review_count
    4. Jakość oferty : review_scores_rating, review_scores_rating, review_scores_cleanliness, review_scores_checkin, review_scores_communication, review_scores_location,review_scores_location
    5. Cechy hosta: host_is_superhost

3. Podział danych
    zbiór treningowy – 80%
    zbiór walidacyjny – 10%
    zbiór testowy – 10%

4. Modele
    Ridge Regression (model liniowy)
        - model bazowy
        - wykorzystuje kodowanie One-Hot dla cech kategorycznych
        - trenowany na log(1 + base_price) w celu stabilizacji rozkładu cen

    CatBoost Regressor (model nieliniowy)
        - modeluje nieliniowe zależności i interakcje
        - trenowany z funkcją straty Quantile (median)
        - zapewnia troche lepszą jakość predykcji niż model liniowy

Model Liniowy Ridge

In [12]:
import pandas as pd
from pathlib import Path

from catboost import CatBoostRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

import numpy as np

BASE_DIR = Path.cwd()
SPLIT_DIR = BASE_DIR / "Processed_data/splits_base_price"


train_df = pd.read_csv(SPLIT_DIR / "train.csv")
val_df = pd.read_csv(SPLIT_DIR / "val.csv")
test_df = pd.read_csv(SPLIT_DIR / "test.csv")

zdefiniowanie zestawu cech wejściowych opisujących ofertę (cechy obiektu, lokalizacji, popularności i jakości) oraz dane są dzielone na macierze cech (X_*) i wektory wartości docelowych (y_*) dla zbioru treningowego, walidacyjnego i testowego

In [13]:
TARGET = "base_price"


FEATURES = [
    "room_type",
    "property_type",
    "accommodates",
    "bedrooms",
    "beds",
    "bathrooms",
    "city",
    "neighbourhood_cleansed",
    "latitude",
    "longitude",
    "minimum_nights",
    "maximum_nights",
    "host_is_superhost",
    "amenities_count",
    "number_of_reviews",
    "reviews_per_month",
    "avg_rating",
    "review_count",
    "review_scores_rating",
    "review_scores_accuracy",
    "review_scores_cleanliness",
    "review_scores_checkin",
    "review_scores_communication",
    "review_scores_location",
    "review_scores_value",
]

# Keep only columns that exist
FEATURES = [c for c in FEATURES if c in train_df.columns]

X_train = train_df[FEATURES].copy()
y_train = train_df[TARGET].astype(float)

X_val = val_df[FEATURES].copy()
y_val = val_df[TARGET].astype(float)

X_test = test_df[FEATURES].copy()
y_test = test_df[TARGET].astype(float)

Usuwanie skrajnych wartości ceny (outliery) ze zbioru treningowego, a następnie przycina wartości w zbiorach walidacyjnym i testowym do tego samego zakresu, aby uniknąć data leakage

In [14]:
p1 = y_train.quantile(0.01)
p99 = y_train.quantile(0.99)

train_mask = (y_train >= p1) & (y_train <= p99)
X_train = X_train.loc[train_mask]
y_train = y_train.loc[train_mask]


y_val = y_val.clip(lower=p1, upper=p99)
y_test = y_test.clip(lower=p1, upper=p99)

Rozdzielenie cechy na kategoryczne i numeryczne oraz definiuje pipeline przetwarzania. Cechy kategoryczne kodowane metodą One-Hot Encoding przed uczeniem modelu.

In [15]:
y_train_log = np.log1p(y_train)
categorical_features = [c for c in ["room_type", "property_type", "city",
                                    "neighbourhood_cleansed"] if c in FEATURES]
numerical_features = [c for c in FEATURES if c not in categorical_features]

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_features),
        ("num", numerical_transformer, numerical_features)
    ],
    remainder="drop"
)

Trenowanie modelu

In [16]:
model = Ridge(alpha=1.0, random_state=42)

pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", model)
])

pipeline.fit(X_train, y_train_log)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['room_type', 'property_type',
                                                   'city',
                                                   'neighbourhood_cleansed']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['accommodates', 'bedrooms',
                                                   '...
                                                   'minimum_nights',
                                                   'maximum_nights',
                                                   'host_is_superhost',
                                                   'amenities_count',
                                                   'number_of_reviews',
                                                   'reviews_per_month',
                                                   'avg_rating', 'review_count',
                                                   'review_scores_rating',
                                                   'review_scores_accuracy',
                                                   'review_scores_cleanliness',
                                                   'review_scores_checkin',
                                                   'review_scores_communication',
                                                   'review_scores_location',
                                                   'review_scores_value'])])),
                ('model', Ridge(random_state=42))])

Testowanie modelu

In [17]:
def smape(y_true, y_pred):
    denom = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    denom = np.where(denom == 0, np.nan, denom)
    return np.nanmean(np.abs(y_true - y_pred) / denom) * 100


def evaluate(name, X, y_true):
    preds_log = pipeline.predict(X)
    preds = np.expm1(preds_log)

    preds = np.clip(preds, p1, p99)

    mae = mean_absolute_error(y_true, preds)

    # Safe MAPE
    y_safe = (y_true.replace(0, np.nan) if hasattr(y_true, "replace")
              else y_true)
    mape = np.nanmean(np.abs((y_safe - preds) / y_safe)) * 100

    smape_val = smape(y_true.to_numpy() if hasattr(y_true, "to_numpy") else
                      y_true, preds)

    print(f"{name}  MAE:{mae:.2f} | MAPE:{mape:.2f}% | SMAPE:{smape_val:.2f}%")


print("RIDGE BASELINE: BASE_PRICE PER LISTING")
evaluate("TRAIN", X_train, y_train)
evaluate("VAL", X_val, y_val)
evaluate("TEST", X_test, y_test)


RIDGE BASELINE: BASE_PRICE PER LISTING
TRAIN  MAE:25.46 | MAPE:7.82% | SMAPE:7.63%
VAL  MAE:26.90 | MAPE:8.89% | SMAPE:8.61%
TEST  MAE:23.83 | MAPE:7.39% | SMAPE:7.31%


Model CatBoost Regressor

Uzupełnienie brakujących wartości w cechach numerycznych medianą, w cechach kategorycznych specjalnym tokenem „unknown”, a następnie wyznacza indeksy cech kategorycznych wymagane przez model CatBoost

In [18]:
num_features = [c for c in FEATURES if c not in categorical_features]
num_imputer = SimpleImputer(strategy="median")
X_train[num_features] = num_imputer.fit_transform(X_train[num_features])
X_val[num_features] = num_imputer.transform(X_val[num_features])
X_test[num_features] = num_imputer.transform(X_test[num_features])

for c in categorical_features:
    X_train[c] = X_train[c].astype(str).fillna("unknown")
    X_val[c] = X_val[c].astype(str).fillna("unknown")
    X_test[c] = X_test[c].astype(str).fillna("unknown")

cat_feature_indices = [FEATURES.index(c) for c in categorical_features]

# Log(price) check
USE_LOG_TARGET = False
y_train_fit = np.log1p(y_train) if USE_LOG_TARGET else y_train
y_val_fit = np.log1p(y_val) if USE_LOG_TARGET else y_val

Trenowanie modelu

In [19]:
model = CatBoostRegressor(
    loss_function="Quantile:alpha=0.5",
    iterations=2500,
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=10,
    min_data_in_leaf=20,
    random_seed=42,
    verbose=200,
    od_type="Iter",
    od_wait=150
)

model.fit(
    X_train,
    y_train_fit,
    cat_features=cat_feature_indices,
    eval_set=(X_val, y_val_fit),
    use_best_model=True
)

0:	learn: 67.8665697	test: 64.7110267	best: 64.7110267 (0)	total: 151ms	remaining: 6m 16s
200:	learn: 7.8232445	test: 10.4861990	best: 10.4861990 (200)	total: 1.06s	remaining: 12.1s
400:	learn: 5.2932520	test: 9.1574668	best: 9.1574668 (400)	total: 1.87s	remaining: 9.79s
600:	learn: 4.3809294	test: 8.9428392	best: 8.9428392 (600)	total: 2.5s	remaining: 7.89s
800:	learn: 3.7911279	test: 8.7497567	best: 8.7355953 (788)	total: 3.19s	remaining: 6.76s
1000:	learn: 3.3904525	test: 8.6164214	best: 8.6092585 (964)	total: 3.81s	remaining: 5.71s
1200:	learn: 3.0923425	test: 8.5739464	best: 8.5577437 (1156)	total: 4.46s	remaining: 4.82s
1400:	learn: 2.8382163	test: 8.5281416	best: 8.5068267 (1348)	total: 5.08s	remaining: 3.98s
1600:	learn: 2.6633749	test: 8.5021656	best: 8.4914788 (1476)	total: 5.75s	remaining: 3.23s
Stopped by overfitting detector  (150 iterations wait)

bestTest = 8.491478759
bestIteration = 1476

Shrink model to first 1477 iterations.


Testowanie modelu

In [20]:
def evaluate(name, X, y_true):
    preds = model.predict(X)
    if USE_LOG_TARGET:
        preds = np.expm1(preds)

    preds = np.clip(preds, p1, p99)

    mae = mean_absolute_error(y_true, preds)

    y_safe = (y_true.replace(0, np.nan) if hasattr(y_true, "replace")
              else y_true)
    mape = np.nanmean(np.abs((y_safe - preds) / y_safe)) * 100

    print(f"{name} | MAE: {mae:.2f} | MAPE: {mape:.2f}%")


print("CATBOOST BASE_PRICE ")
evaluate("TRAIN", X_train, y_train)
evaluate("VAL", X_val, y_val)
evaluate("TEST", X_test, y_test)

CATBOOST BASE_PRICE 
TRAIN | MAE: 7.20 | MAPE: 2.40%
VAL | MAE: 16.98 | MAPE: 5.53%
TEST | MAE: 13.09 | MAPE: 4.26%


Wnioski: Model Ridge zapewnia stabilne, ale ograniczone wyniki (MAE ≈ 24–27 USD, MAPE ≈ 7–9%), co potwierdza jego rolę jako solidnego modelu bazowego o dobrej generalizacji.
Model CatBoost znacząco przewyższa Ridge, redukując błąd ponad dwukrotnie (MAE ≈ 13 USD, MAPE ≈ 4% na zbiorze testowym), co wskazuje na skuteczne uchwycenie nieliniowych zależności między cechami i lepsze wykorzystanie informacji kategorycznych oraz ocen użytkowników